# Using Pandas to interrogate a csv


WE have a particular column called "cast" that appears to a be a list of dictionaries. I need to move that into its own csv so I can clean the dwguide.csv.

I REALLY struggled, but ChatGPT solved it. A fail, I know, but it works.


In [1]:
import pandas as pd
import json

# Load your CSV file into a Pandas DataFrame
df = pd.read_csv('dwguide.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   episodenbr     851 non-null    int64  
 1   title          851 non-null    object 
 2   weekday        851 non-null    object 
 3   broadcastdate  851 non-null    object 
 4   broadcasthour  851 non-null    object 
 5   duration       851 non-null    object 
 6   views          851 non-null    object 
 7   share          156 non-null    object 
 8   AI             626 non-null    float64
 9   chart          851 non-null    int64  
 10  cast           851 non-null    object 
 11  crew           851 non-null    object 
 12  summary        757 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 86.6+ KB


In [2]:

# Define a function to safely parse JSON
def parse_json(json_str):
    try:
        return json.loads(json_str)
    except (ValueError, TypeError):
        return []

# Apply the function to the column containing JSON data
df['cast'] = df['cast'].apply(parse_json)

# Create a new DataFrame in long format
long_format_data = []

for index, row in df.iterrows():
    episodenbr = row['episodenbr']
    for item in row['cast']:
        role = item.get('role', 'N/A')
        name = item.get('name', 'N/A')
        long_format_data.append({'episodenbr': episodenbr, 'role': role, 'name': name})

long_df = pd.DataFrame(long_format_data)


long_df.to_csv('castlist.csv', index=False) # this creates the castlist cav with Episodenbr as the index



So to clean it further I will remove the cast column from the df, and discard the crew one too as I do not need it.

In [3]:
df.info() #before

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   episodenbr     851 non-null    int64  
 1   title          851 non-null    object 
 2   weekday        851 non-null    object 
 3   broadcastdate  851 non-null    object 
 4   broadcasthour  851 non-null    object 
 5   duration       851 non-null    object 
 6   views          851 non-null    object 
 7   share          156 non-null    object 
 8   AI             626 non-null    float64
 9   chart          851 non-null    int64  
 10  cast           851 non-null    object 
 11  crew           851 non-null    object 
 12  summary        757 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 86.6+ KB


In [4]:
del df['cast']

del df['crew']

df.info() #after

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   episodenbr     851 non-null    int64  
 1   title          851 non-null    object 
 2   weekday        851 non-null    object 
 3   broadcastdate  851 non-null    object 
 4   broadcasthour  851 non-null    object 
 5   duration       851 non-null    object 
 6   views          851 non-null    object 
 7   share          156 non-null    object 
 8   AI             626 non-null    float64
 9   chart          851 non-null    int64  
 10  summary        757 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 73.3+ KB


So let's write this as dwguide_clean.csv so we don't have to go through this rigmarole in the future, and the messy json columns are gone.

In [5]:
df.to_csv('dwguide_clean.csv', index=False)